# Using the cache

First make sure that the dependencies are installed:

```bash
pip install -r requirements-dev.txt
```

Then run redis using docker compose:

```bash
docker compose -f docker-compose.redis.yml up -d
```

Then run the notebook. 

If you want to view the data in redis, you can use the redis-cli:

```bash
brew install redis
redis-cli -p 6380
```

In [1]:
import fsspec
import xarray as xr
from redis_fsspec_cache.sync import RedisCachingFileSystem

fs = fsspec.filesystem(
    "s3",
    anon=True,
)

cached_fs = RedisCachingFileSystem(
    redis_host="localhost",
    redis_port=6380,
    expiry_time=60,
    method='chunk',
    fs=fs,
)

Let try to access a single chunk from a remote GRIB file we know about in an s3 bucket:

```json
"gust\/0.0.0": [
    "s3:\/\/noaa-hrrr-bdp-pds\/hrrr.20230927\/conus\/hrrr.t00z.wrfsubhf00.grib2",
    2884409,
    1133540
],
```

In [2]:
url = "s3://noaa-hrrr-bdp-pds/hrrr.20230927/conus/hrrr.t00z.wrfsubhf00.grib2"
offset = 2884409
length = 1133540

In [4]:
%%time

nocache_chunk = fs.read_block(url, offset=offset, length=length)

CPU times: user 150 ms, sys: 110 ms, total: 260 ms
Wall time: 1.44 s


In [5]:
%%time

uncached_chunk = cached_fs.read_block(url, offset=offset, length=length)

CPU times: user 15.6 ms, sys: 5.97 ms, total: 21.6 ms
Wall time: 338 ms


In [6]:
%%time

cached_chunk = cached_fs.read_block(url, offset=offset, length=length)

CPU times: user 5.68 ms, sys: 4.53 ms, total: 10.2 ms
Wall time: 58.1 ms


In [7]:
is_correct_length = len(cached_chunk) == length
is_correct_data = cached_chunk == uncached_chunk and cached_chunk == nocache_chunk
print(f"Correct length: {is_correct_length}")
print(f"Correct data: {is_correct_data}")

Correct length: True
Correct data: True


In [2]:
%%time
f = cached_fs.open('s3://noaa-nos-ofs-pds/dbofs/netcdf/202405/nos.dbofs.fields.f001.20240503.t18z.nc', 'rb')
ds = xr.open_dataset(f, chunks={}, engine='h5netcdf')
ds


CPU times: user 792 ms, sys: 363 ms, total: 1.16 s
Wall time: 515 ms


<xarray.Dataset> Size: 32MB
Dimensions:        (tracer: 2, boundary: 4, s_rho: 10, s_w: 11, eta_rho: 732,
                    xi_rho: 119, eta_u: 732, xi_u: 118, eta_v: 731, xi_v: 119,
                    eta_psi: 731, xi_psi: 118, ocean_time: 1)
Coordinates:
  * s_rho          (s_rho) float64 80B -0.95 -0.85 -0.75 ... -0.25 -0.15 -0.05
  * s_w            (s_w) float64 88B -1.0 -0.9 -0.8 -0.7 ... -0.3 -0.2 -0.1 0.0
    lon_rho        (eta_rho, xi_rho) float64 697kB dask.array<chunksize=(732, 119), meta=np.ndarray>
    lat_rho        (eta_rho, xi_rho) float64 697kB dask.array<chunksize=(732, 119), meta=np.ndarray>
    lon_u          (eta_u, xi_u) float64 691kB dask.array<chunksize=(732, 118), meta=np.ndarray>
    lat_u          (eta_u, xi_u) float64 691kB dask.array<chunksize=(732, 118), meta=np.ndarray>
    lon_v          (eta_v, xi_v) float64 696kB dask.array<chunksize=(731, 119), meta=np.ndarray>
    lat_v          (eta_v, xi_v) float64 696kB dask.array<chunksize=(731, 119), meta=np.ndarray>
    lon_psi        (eta_psi, xi_psi) float64 690kB dask.array<chunksize=(731, 118), meta=np.ndarray>
    lat_psi        (eta_psi, xi_psi) float64 690kB dask.array<chunksize=(731, 118), meta=np.ndarray>
  * ocean_time     (ocean_time) datetime64[ns] 8B 2024-05-03T19:00:00
Dimensions without coordinates: tracer, boundary, eta_rho, xi_rho, eta_u, xi_u,
                                eta_v, xi_v, eta_psi, xi_psi
Data variables: (12/79)
    ntimes         int32 4B ...
    ndtfast        int32 4B ...
    dt             float64 8B ...
    dtfast         float64 8B ...
    dstart         datetime64[ns] 8B ...
    nHIS           int32 4B ...
    ...             ...
    w              (ocean_time, s_w, eta_rho, xi_rho) float32 4MB dask.array<chunksize=(1, 6, 732, 119), meta=np.ndarray>
    temp           (ocean_time, s_rho, eta_rho, xi_rho) float32 3MB dask.array<chunksize=(1, 10, 732, 119), meta=np.ndarray>
    salt           (ocean_time, s_rho, eta_rho, xi_rho) float32 3MB dask.array<chunksize=(1, 10, 732, 119), meta=np.ndarray>
    Pair           (ocean_time, eta_rho, xi_rho) float32 348kB dask.array<chunksize=(1, 732, 119), meta=np.ndarray>
    Uwind          (ocean_time, eta_rho, xi_rho) float32 348kB dask.array<chunksize=(1, 732, 119), meta=np.ndarray>
    Vwind          (ocean_time, eta_rho, xi_rho) float32 348kB dask.array<chunksize=(1, 732, 119), meta=np.ndarray>
Attributes: (12/33)
    file:              nos.dbofs.fields.forecast.20240503.t18z_0002.nc
    format:            netCDF-4/HDF5 file
    Conventions:       CF-1.4, SGRID-0.3
    type:              ROMS/TOMS history file
    title:             dbofs forecast RUN in operational mode
    var_info:          varinfo.yaml
    ...                ...
    compiler_command:  /opt/cray/pe/craype/2.7.10/bin/ftn
    compiler_flags:    -fp-model precise -ip -O3
    tiling:            008x016
    history:           ROMS/TOMS, Version 4.1, Friday - May 3, 2024 -  6:49:2...
    ana_file:          ROMS/Functionals/ana_btflux.h, ROMS/Functionals/ana_ra...
    CPP_options:       mode, ADD_FSOBC, ADD_M2OBC, ANA_BSFLUX, ANA_BTFLUX, AN...

In [3]:
%%time 
len(ds.lat_u.values)

CPU times: user 110 ms, sys: 10 ms, total: 120 ms
Wall time: 130 ms


732

In [4]:
%%time 
len(ds.lat_u.values)

CPU times: user 4.71 ms, sys: 2.78 ms, total: 7.49 ms
Wall time: 12.8 ms


732